### Install tendb
Must install latest tendb before running the commands
Install from released package from pypi
```bash
$ pip install -i https://test.pypi.org/simple/ tendb
```
Read local setup.py and install tendb
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/tendb
```
Install from local wheel file
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/tendb/dist/tendb-0.0.1-py3-none-any.whl
```

Read Arrow table

In [1]:
import pyarrow as pa
from pyarrow import csv

In [2]:
pa.show_versions()

pyarrow version info
--------------------
Package kind: not indicated
Arrow C++ library version: 2.0.0
Arrow C++ compiler: GNU 7.5.0
Arrow C++ compiler flags: -fvisibility-inlines-hidden -std=c++17 -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/hkverma/miniconda3/envs/tendb/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/arrow-cpp-ext_1604401055323/work=/usr/local/src/conda/arrow-cpp-2.0.0 -fdebug-prefix-map=/home/hkverma/miniconda3/envs/tendb=/usr/local/src/conda-prefix -fdiagnostics-color=always -fuse-ld=gold -O3 -DNDEBUG
Arrow C++ git revision: 4d367ccc36657519870e6efafc537e5c287fe5f1
Arrow C++ git description: 


In [3]:
csv_options = pa.csv.ParseOptions(delimiter='|')
lineitem_table = pa.csv.read_csv(input_file='/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/lineitem.tbl', parse_options=csv_options)
lineitem_table

pyarrow.Table
L_ORDERKEY: int64
L_PARTKEY: int64
L_SUPPKEY: int64
L_LINENUMBER: int64
L_QUANTITY: int64
L_EXTENDEDPRICE: double
L_DISCOUNT: double
L_TAX: double
L_RETURNFLAG: string
L_LINESTATUS: string
L_SHIPDATE: timestamp[s]
L_COMMITDATE: timestamp[s]
L_RECEIPTDATE: timestamp[s]
L_SHIPINSTRUCT: string
L_SHIPMODE: string
L_COMMENT: string

In [4]:
len(lineitem_table)

6001215

In [5]:
isinstance(lineitem_table, object)

True

In [6]:
hex(id(lineitem_table))

'0x7fcd382e2730'

In [7]:
lineitem_table.num_rows

6001215

In [8]:
lineitem_table.num_columns

16

TenDB sample commands

In [9]:
import tendb as ten

<frozen importlib._bootstrap>:228: RuntimeWarning: compiletime version 3.6 of module 'tendb._tendb' does not match runtime version 3.9
<frozen importlib._bootstrap>:228: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 864 from C header, got 880 from PyObject


In [10]:
tb = ten._tendb.CTenDB()

Added a new cache


In [11]:
tb.show_versions()

'0.0.2'

In [12]:
isinstance(lineitem_table, pa.Table)

True

In [13]:
tb.add_table(b'lineitem', lineitem_table)

b'lineitem'

In [14]:
tb.query6()

156594095.60960016